In [1]:
# Filter warnings for readability
import warnings
warnings.filterwarnings('ignore')

Imports

In [2]:
import pandas as pd
from transformers import pipeline
import torch
from wordcloud import WordCloud
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

W0709 00:36:37.990000 52756 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Read Preprocessed Data

In [3]:
preprocessed_df = pd.read_csv('preprocessed_df.csv')

In [4]:
preprocessed_df.head()

,title
0,DSWD DROMIC Report on the Tornado Incident in ...
1,New state report : Wildfire smoke increased de...
2,New Year Day flood alerts issued in Cambridges...
3,Gold Coast flash flooding danger : Weather ale...
4,"Almost 100 , 000 ordered to leave area after J..."


In [5]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27533 entries, 0 to 27532
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   27533 non-null  object
dtypes: object(1)
memory usage: 215.2+ KB


### __Step 1__: Build a *Multi-Label Zero-Shot-Classifier*

In [ ]:
labels = ["Earthquake", "Floods", "Volcano", "Tornado", "Wildfire"]

if torch.cuda.is_available():
    print("Using GPU")
    classifier = pipeline("zero-shot-classification", model=config['zero_shot_model_gpu'], device=0) # Use GPU
    results = classifier(preprocessed_df['title'].tolist()[:100], candidate_labels=labels, batch_size=config['batch_size_gpu'])
else:
    print("Using CPU")
    classifier = pipeline("zero-shot-classification", model=config['zero_shot_model_cpu'], device=-1) # Use CPU
    results = classifier(preprocessed_df['title'].tolist(), candidate_labels=labels, batch_size=config['batch_size_cpu'])

Using GPU


Example Result

In [ ]:
results[0]

In [8]:
# Free up CUDA cores:
del classifier
del results
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

NameError: name 'classifier' is not defined

In [ ]:
parsed = [
    {
        "natural_catastrophe_event": r["labels"][0],
        "zero_shot_score": r["scores"][0]
    }
    for r in results
]

In [ ]:
preprocessed_df[["natural_catastrophe_event", "zero_shot_score"]] =  pd.DataFrame(parsed)

In [ ]:
preprocessed_df.head()

### Post-Modelling Analysis

In [ ]:
preprocessed_df['natural_catastrophe_event'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: Set a clean style
sns.set(style="whitegrid")

# Plot
plt.figure(figsize=(10, 6))
sns.countplot(
    data=preprocessed_df, 
    x="natural_catastrophe_event", 
    order=preprocessed_df["natural_catastrophe_event"].value_counts().index,
    palette="Set2"
)

# Add labels and title
plt.title("Distribution of Predicted Natural Catastrophe Events", fontsize=16)
plt.xlabel("Event Type", fontsize=12)
plt.ylabel("Number of Articles", fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
sns.set(style="whitegrid")

# Create the boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=preprocessed_df,
    x="natural_catastrophe_event",
    y="zero_shot_score",
    palette="Set3"
)

# Labeling
plt.title("Zero-Shot Score Distribution by Natural Catastrophe Event", fontsize=16)
plt.xlabel("Natural Catastrophe Event", fontsize=12)
plt.ylabel("Zero-Shot Score", fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(preprocessed_df["zero_shot_score"], shade=True, color="purple")

plt.title("Density of Zero-Shot Classification Scores", fontsize=16)
plt.xlabel("Zero-Shot Score", fontsize=12)
plt.ylabel("Density", fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filter titles with low confidence
low_score_titles = preprocessed_df[preprocessed_df["zero_shot_score"] < 0.4]["title"]

# Combine all titles into one string
text_blob = " ".join(low_score_titles.astype(str).tolist())

# Generate word cloud
wordcloud = WordCloud(width=1000, height=500, background_color="white", colormap="viridis").generate(text_blob)

# Plot
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Titles with Zero-Shot Score < 0.4", fontsize=16)
plt.tight_layout()
plt.show()
